## Parte 3 - Classificação - Otimizações

---
<img align="center" width="150" src="https://logodownload.org/wp-content/uploads/2017/05/ifood-logo-0.png">

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd

## Lendos os Dados

In [3]:
df_abt_train = pd.read_csv('/content/drive/MyDrive/projeto-ia-datasets/ifood/abt_classificacao_ifood_train.csv')
df_abt_train.head(2)

,data_ref,customer_id,uf,receita_1m,qtd_pedidos_1m,recencia,churn_next_month
0,2019-07-01 00:00:00+00:00,0001a8e61d8b08ad436e8e6f4adeb399b88df962c72d9d...,PR,102.0,2,15,1
1,2019-07-01 00:00:00+00:00,0001a9f97d01d2696cf70c7657ee2d039388d691720ff9...,SP,52.0,1,19,1


In [4]:
df_abt_train.data_ref.unique()

array(['2019-07-01 00:00:00+00:00'], dtype=object)

In [5]:
df_abt_oot = pd.read_csv('/content/drive/MyDrive/projeto-ia-datasets/ifood/abt_classificacao_ifood_oot.csv')
df_abt_oot.head(2)

,data_ref,customer_id,uf,receita_1m,qtd_pedidos_1m,recencia,churn_next_month
0,2019-08-01 00:00:00+00:00,000a1fac4f7a67cc3f2e7667167597cb2c9a1b9edafe18...,SP,85.89,3,11,0
1,2019-08-01 00:00:00+00:00,0014b7013c66a05d0b5ce0687d614ac220d3ae1af398d2...,PE,211.96,5,11,0


In [6]:
df_abt_oot.data_ref.unique()

array(['2019-08-01 00:00:00+00:00'], dtype=object)

In [7]:
key_vars = ['data_ref', 'customer_id']
num_vars = ['receita_1m', 'qtd_pedidos_1m', 'recencia']
cat_vars = ['uf']
target = 'churn_next_month'
features = cat_vars + num_vars

In [8]:
# Dados de treinamento somente com as variáveis numéricas
X_train = df_abt_train[features]
y_train = df_abt_train[target]

# Dados de avaliação (out of time) somente com as variáveis numéricas
X_oot = df_abt_oot[features]
y_oot = df_abt_oot[target]

In [9]:
print(X_train.shape, y_train.shape)

(30079, 4) (30079,)


In [10]:
print(X_oot.shape, y_oot.shape)

(18441, 4) (18441,)


## Carregando os pacotes

In [11]:
!pip install lightgbm feature_engine catboost

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.pipeline import Pipeline

from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer
from feature_engine.encoding import OneHotEncoder

from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper

from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

# Exercícios

## Q1 - Comparando os algoritmos

Crie um DataFrame com as métricas de treino para os seguintes algoritmos:

- Regressão Logistica (`logistic_regression`)
- DecisionTree (`dt`)
- RandomForest (`random_forest`)
- LightGBM (`lgbm`)
- CatBoost (`catboost`)
- ExtremeGradientBoost (`xgb`)


- Utilize o parametro `random_state=42` para os algoritmos.
- Utilize a função `cross_validate` com os seguintes parâmetros:
    - `scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc']`
    - `cv=5`
    - `n_jobs=-1`

- Lembre-se de aplicar as etapas de pre-processamento adequadas.

- A tabela abaixo é um exemplo da saída esperada.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>acc</th>
      <th>precision</th>
      <th>recall</th>
      <th>f1</th>
      <th>roc_auc</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>lgbm</th>
      <td>0.690482</td>
      <td>0.594210</td>
      <td>0.630866</td>
      <td>0.611957</td>
      <td>0.753395</td>
    </tr>
    <tr>
      <th>catboost</th>
      <td>0.689584</td>
      <td>0.593740</td>
      <td>0.626139</td>
      <td>0.609504</td>
      <td>0.752277</td>
    </tr>
    <tr>
      <th>logistic_regression</th>
      <td>0.690515</td>
      <td>0.607244</td>
      <td>0.566680</td>
      <td>0.586223</td>
      <td>0.750642</td>
    </tr>
    <tr>
      <th>xgb</th>
      <td>0.683833</td>
      <td>0.590656</td>
      <td>0.595722</td>
      <td>0.593166</td>
      <td>0.743513</td>
    </tr>
    <tr>
      <th>random_forest</th>
      <td>0.642142</td>
      <td>0.540034</td>
      <td>0.506443</td>
      <td>0.522644</td>
      <td>0.688558</td>
    </tr>
    <tr>
      <th>dt</th>
      <td>0.615246</td>
      <td>0.502968</td>
      <td>0.491923</td>
      <td>0.497342</td>
      <td>0.600950</td>
    </tr>
  </tbody>
</table>

**OS VALORES PODEM SER DIFERENTES**

In [13]:
dt = Pipeline(steps=[
    ("numeric_imputer", MeanMedianImputer(variables=num_vars, imputation_method="median")),
    ("categorical_imputer", CategoricalImputer(variables=cat_vars, fill_value="missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("dt", DecisionTreeClassifier(random_state=42))
])

In [14]:
rf = Pipeline(steps=[
    ("numeric_imputer",MeanMedianImputer(variables=num_vars, imputation_method="median")),
    ("categorical_imputer", CategoricalImputer(variables=cat_vars, fill_value="missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("rf", RandomForestClassifier(random_state=42))
])

In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [18]:
cross_val_dt = cross_validate(dt, X_train,y_train,
                              scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], cv= skf,n_jobs=-1)

In [19]:
pd.DataFrame(cross_val_dt).mean()

fit_time          0.655803
score_time        0.193719
test_accuracy     0.614050
test_precision    0.501230
test_recall       0.494933
test_f1           0.498024
test_roc_auc      0.599544
dtype: float64

In [20]:
cross_val_rf = cross_validate(rf, X_train,y_train,
                              scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'], cv= skf,n_jobs=-1)

In [21]:
pd.DataFrame(cross_val_rf).mean()

fit_time          6.113528
score_time        0.645034
test_accuracy     0.637189
test_precision    0.533140
test_recall       0.500689
test_f1           0.516361
test_roc_auc      0.687022
dtype: float64

## Q2 - Avaliando na base OOT

Para o modelo que teve o melhor desempenho na validação cruzada, gere novamente o pipeline com os respectivos steps de preprocessamento e o algoritmo escolhido. Salve o modelo treinado variável `melhor_modelo`.

Lembre-se de definir o `random_state=42`.

Calcule as seguintes métricas:
- `roc_auc_score`
- `accuracy_score`

In [ ]:
[:,1]

## Q3 - Otimização dos hiperparâmetros

Faça a otimização dos hiperparâmetros do melhor modelo que foi encontrado. Utilize a técnica de `RandomizedSearchCV`.

Utilize a variável `melhor_modelo` para otimizar.

Os parâmetros para a técnica `RandomizedSearchCV` deve ser:
- `scoring='roc_auc', cv=3, n_iter=6, n_jobs=-1, verbose=1, random_state=42`

Escolha pelo menos 2 hiperparâmetros e pelo menos 3 valores para cada hiperparâmetro.

Lembre-se de definir o `random_state=42`.

Salve o resultado da otimização (modelo já fitado com a melhor combinação de hiperparâmetros) na variável `melhor_modelo_otimizado`.

Calcule as seguintes métricas:
- `roc_auc_score`
- `accuracy_score`

In [ ]:
rand = Ra  (dfds , )

## Q4 - Feature Selection - RFE

Utilize a tecnica de RFE para selecionar automaticamente as 5 principais variáveis.

Recupere os melhores parametros que foram encontrados nas otimizações realizadas na questão anterior.

Crie o pipeline novamente com as etapas de preprocessamento, a seleção de features e o melhor algoritmos.

Salve o resultado da aplicação da técnica RFE na variável `melhor_modelo_otimizado_rfe`.

Calcule as seguintes métricas:
- `roc_auc_score`
- `accuracy_score`

## Q5 - Avaliando o melhor modelo

Dentre os 3 modelos gerados (`melhor_modelo`, `melhor_modelo_otimizado`, `melhor_modelo_otimizado_rfe`) selecione o que apresentou a melhor métrica de `roc_auc_score` e:

- Plot a curva RoC
- Plot a matriz de confusão
- Imprima o `classification_report`
- Calcule as métricas `balanced_accuracy_score` e `accuracy_score`.

Por fim, faça uma análise dos resultados obtidos e do custo operacional desse modelo, para isso considere as seguintes premissas:

- Para os clientes que foram identificados como Churn, o ifood irá disponibilizar um cupom de 7 reais.
- A comissão que o ifood ganha por venda é 20%.


Faça as devidas observações e considerações dos resultados obtidos e como podemos melhorar os resultados de negócio e de performance do modelo.

**ADICIONE SUA RESPOSTA AQUI**



## Q6 - Salvando o melhor modelo

Salve o melhor modelo encontrado na pasta `/content/drive/MyDrive/modelos/`.

In [ ]:
!pip install joblib

In [ ]:
import joblib

In [ ]:
!mkdir '/content/drive/MyDrive/modelos'

In [ ]:
import datetime

date = datetime.datetime.now().strftime("%Y%m%d")
date

In [ ]:
caminho = f'/content/drive/MyDrive/modelos/{date}_ifood_churn_melhor_modelo_otimizado.joblib'
joblib.dump(melhor_modelo, caminho)

# Exercícios complementares

Os exercícios abaixo são complementares para entender como o melhor modelo criado anteriormente irá performar em uma base de produção.

## Q7 - Criando uma nova safra

Crie uma nova safra 2019-09-01 utilizando os dados do mês anterior, que corresponde ao interva-lo de `2019-08-01` (incluido) até `2019-09-01` (não incluído).

- As colunas devem ser as mesmas da base de treinamento e oot.
- Crie e salve o dataframe na variável `df_producao`.
- Defina as váriaveis `key_vars, num_vars, cat_vars, features, X_producao`

<img align="left" width="20" src="http://dinomagri.com/imgs/checkpoint.png" /> &nbsp; **CHECKPOINT** - O DataFrame `df_producao` deverá conter exatamente `(17319, 6)`.

**Perceba que não temos o target, pois essa base é de produção (evento de interesse ainda não ocorreu).**

## Q8 - Carregando o modelo

Carregue o modelo salvo anteriormente na variável `model`.

## Q9 - Realizando as predições

Faça as predições da classe e das respectivas probabilidades (classe 1 e classe 0) e salve nas variáveis

- `y_pred_producao` - deve contar a classe (0 ou 1)
- `y_proba_producao_1` - deve conter a probabilidade de ser 1
- `y_proba_producao_0` - deve conter a probabilidade de ser 0

Por fim, crie 3 novas colunas no `df_producao` para armazenar os valores computados anteriormente.

- A coluna `classe_prevista` deverá conter os valores calculados e salvos na variável `y_pred_producao`.
- A coluna `score_1` deverá conter os valores calculados e salvos na variável `y_proba_producao_1`.
- A coluna `score_0` deverá conter os valores calculados e salvos na variável `y_proba_producao_0`.

## Q10 - Monitoramento de modelos

**Como saber se o modelo irá performar bem ou não em novos dados?**

- Utilizando a biblioteca [Evidently](https://github.com/evidentlyai/evidently). Faça uma analise para identificar se aconteceu DataDrift.

- Utilize como base de referencia a base de treinamento (`df_abt_train`).

- Faça as devidas observações e considerações dos resultados obtidos e como podemos melhorar os resultados de negócio e de performance do modelo.

In [ ]:
!pip install evidently

**ADICIONAR SUA RESPOSTA AQUI**
